# About: Docker Swarmの準備

構築済みのVM上に、DockerEngineをインストールし、Docker Swarmを準備する。

# 設定情報

このNotebookで行う設定は、以下のようにする。

- Docker関連のディレクトリ （コンテナデータ、イメージ、一時ファイル用ディレクトリ等）
  - NIIのベアメタルマシンを想定する場合 ... Docker関係のディレクトリは `/mnt`に配置する
  - それ以外 ... ストレージサイズ等を考慮して、適切なパスを設定すること
- プライベートレジストリ
  - 公開のレジストリのみ使用する場合 ... なし
  - クラウド運用チームで利用する場合 ... Dockerのプライベートレジストリを使用するので、プライベートレジストリのホスト情報を明示する

docker_optsの定義方法は[DAEMON CONFIGURATION FILE](https://docs.docker.com/engine/reference/commandline/dockerd/#daemon-configuration-file)を参照。

In [ ]:
docker_tmp = '/var/lib/docker/tmp'
docker_base = '/var/lib/docker'
docker_opts ={"data-root": docker_base, "insecure-registries": []}

# Notebookと環境のBinding

Inventory中のgroup名でBind対象ホスト(Docker Engineをインストールしたいホスト)を指示する。


- `hosts_file`: Inventoryファイル
- `target_group`: Ansible Inventory中に定義されているグループ( `[グループ名]` のように`[]`で囲まれた定義 )のうち、CoursewareHub構築に使うグループ名を指定します。
  - NFSサーバー専用のサーバーは対象外
  - NFS専用サーバーを含まないグループ名を指定すること


Ansible Inventory中に定義されているグループ( `[グループ名]` のように`[]`で囲まれた定義 )のうち、CoursewareHub構築に使うグループ名を指定します。

In [ ]:
hosts_file = './cwhtest0001_inventory'
target_group = 'cwhtest0001_nodes' # NFSサーバー専用のサーバーは対象外 NFS専用サーバーを含まないグループ名を指定すること


# 接続確認
%env ANSIBLE_INVENTORY={hosts_file}
!ansible -m ping {target_group}

作業ディレクトリを作成

In [ ]:
import tempfile
temp_dir = tempfile.mkdtemp()
temp_dir

# Binding対象の確認

[Prerequisites](https://docs.docker.com/engine/installation/linux/ubuntulinux/#prerequisites)に示されているとおり、このNotebookを使ってDockerをインストールする対象のホストは、以下の条件を満たしている必要がある。

もし、このインストール手順に失敗したら、**この条件を満たす状態にマシンを戻す(パッケージの削除, マシンの再プロビジョニングなど)**ことで、**(このNotebookによって)Dockerをインストール可能な状態に戻す**ことができる。

## 64bit版を使う

Docker Engineを動作させるには64bit版が必要。

In [ ]:
!ansible -a 'uname -m' {target_group}

## kernel versionは最低3.10

3.10未満の古いバージョンの場合はDockerの機能の一部が使えなかったり、データロストやpanicを生じる可能性がある。

そのため、以下のバージョン表示が3.10以上であることを確認しておく。

In [ ]:
!ansible -a 'uname -r' {target_group}

## CentOSは7を想定



In [ ]:
!ansible -b -m yum -a 'name=redhat-lsb' {target_group}

In [ ]:
!ansible -a 'lsb_release -a' {target_group}

## 古いパッケージの削除

古いパッケージの削除

In [ ]:
!ansible -b -m yum -a 'name=docker,docker-client,docker-client-latest,docker-common,docker-latest,docker-latest-logrotate,docker-logrotate,docker-selinux,docker-engine-selinux,docker-engine state=absent' {target_group}

# Docker Engineのインストール

Bind対象にDocker Engineをインストールする。

## リポジトリの設定

In [ ]:
!ansible -b -m yum -a 'name=yum-utils,device-mapper-persistent-data,lvm2' {target_group}

In [ ]:
!ansible -b -a 'yum-config-manager --add-repo https://download.docker.com/linux/centos/docker-ce.repo' {target_group}

## パッケージのインストール

`docker-ce` パッケージをインストールする。

In [ ]:
!ansible -b -m yum -a 'name=docker-ce' {target_group}

## Docker Engineの設定変更

あらかじめ定義した設定情報にしたがい、Docker Engineに与えるDefault Configを指定する。

In [ ]:
import os
import json
with open(os.path.join(temp_dir, 'daemon.json'), 'w') as f:
    f.write(json.dumps(docker_opts))
!cat {temp_dir}/daemon.json

In [ ]:
import os
with open(os.path.join(temp_dir, 'tmpdir.conf'), 'w') as f:
    f.write('''# Systemd drop-in configuration for Docker
[Service]
Environment="DOCKER_TMPDIR={docker_tmp}"'''.format(docker_tmp=docker_tmp))
!cat {temp_dir}/tmpdir.conf

ローカルに作った configファイル を、Bind対象の/etc/default/dockerにコピーし、Docker Engineに反映する。

In [ ]:
!ansible -b -m file -a 'path=/etc/docker state=directory' {target_group}
!ansible -b -m copy -a 'src={temp_dir}/daemon.json dest=/etc/docker/daemon.json' {target_group}
!ansible -b -m file -a 'path=/etc/systemd/system/docker.service.d state=directory' {target_group}
!ansible -b -m copy -a 'src={temp_dir}/tmpdir.conf dest=/etc/systemd/system/docker.service.d/tmpdir.conf' {target_group}

!ansible -b -m file -a 'path={docker_tmp} state=directory' {target_group}
!ansible -b -a 'systemctl daemon-reload' {target_group}
!ansible -b -m service -a 'name=docker state=restarted enabled=yes' {target_group}

念のため、Docker Engineにより /mnt/docker, /mnt/docker-tmp (変更していればそのディレクトリ) にファイルが作成されていることを確認する。

In [ ]:
!ansible -b -a 'ls -la {docker_tmp} {docker_base}' {target_group}

サービスの状態を確認

In [ ]:
!ansible -b -a 'systemctl status docker' {target_group}

マシンのboot時にdockerサービスが起動するようにしておく。

In [ ]:
!ansible -b -a 'systemctl enable docker' {target_group}

Docker Engineのバージョンを確認する。

In [ ]:
!ansible -b -a 'docker version' {target_group}

Docker Engineの設定状況も確認しておく。

In [ ]:
!ansible -b -a 'docker info' {target_group}

# Docker Composeのインストール

*2022/2/17時点* では、docker-composeのv1系のバージョンは`1.29.2`となる。

In [ ]:
!ansible -b -m shell \
         -a 'curl -L https://github.com/docker/compose/releases/download/1.29.2/docker-compose-`uname -s`-`uname -m` \
                 > /usr/local/bin/docker-compose' {target_group}

In [ ]:
!ansible -b -a 'chmod +x /usr/local/bin/docker-compose' {target_group}

In [ ]:
!ansible -b -a '/usr/local/bin/docker-compose --version' {target_group}

# Docker Engineの動作確認

まずはお試しで、hello-worldイメージを実行してみる。`Hello from Docker`のようなメッセージが表示されたらOK。

In [ ]:
!ansible -b -a 'docker run hello-world' {target_group}

Dockerのhello-worldイメージが実行された。OK。

# Docker Composeの動作確認

Docker Composeが動作することも確認しておく。

まずローカルにdocker-compose.ymlファイルを準備。

In [ ]:
!mkdir -p {temp_dir}/hello-compose/

In [ ]:
%%writefile {temp_dir}/hello-compose/docker-compose.yml
version: '2'
services:
  test-hello-world:
    image: hello-world

作成したdocker-compose.ymlを、Bind対象ホストにアップロードする。

In [ ]:
!ansible -b -m copy -a 'src={temp_dir}/hello-compose dest=~' {target_group}

実行してみる。`Hello from Docker`のようなメッセージが表示されたらOK。

In [ ]:
!ansible -b -a 'chdir=~/hello-compose /usr/local/bin/docker-compose up' {target_group}

# Docker Swarmの準備

Swarm Modeを有効化する。

https://docs.docker.com/engine/swarm/swarm-tutorial/create-swarm/

## 前提条件の確認

Docker Swarmは、Overlay Networkを使用するので、以下の前提条件が満たされていることを確認する。

https://docs.docker.com/network/overlay/ 

の、Create an overlay networkの、Prerequisitesが満たされているか確認する。


In [ ]:
# 前提条件が満たされていることを確認して、Freezeして先に進める
assert False

## ノードのroleを設定

Swarmのmanager/worker(master/slave)と、Swarmの通信に使用するIPアドレスは、Inventoryから設定する。


masterとするAnsibleのgroup名を指定する。対象が正しいか、Ansible pingモジュールで動作確認する。

In [ ]:
target_master = 'cwhub_master'

!ansible -m ping {target_master} -l {target_group}

workerとするnodeはAnsibleのgroup名で定義する。Ansible pingモジュールで動作確認する。

In [ ]:
target_nodes = 'cwhub_nodes'

!ansible -m ping {target_nodes} -l {target_group}

各々のノードには、servicenet_ip変数が定義されている必要がある。

In [ ]:
!ansible -m debug -a 'msg={{{{ servicenet_ip }}}}' {target_master} -l {target_group}
!ansible -m debug -a 'msg={{{{ servicenet_ip }}}}' {target_nodes} -l {target_group}

## hostsの設定

`/etc/hosts`を設定する。

Swarm Nodeの通信に使うIPアドレスが、互いに名前が解決できるようにしておく。（hostsに登録されてないと名前解決できない環境のための設定）

In [ ]:
!ansible -a hostname {target_group}

inventory上のhostnameと設定されているhostnameの関係を調べる

In [ ]:
import re
ping_pattern = re.compile(r'^(\S+)\s*\|\s*(?:SUCCESS|CHANGED)\s*.*\>\>.*$')
hostname_result = !ansible -a hostname {target_group}
hostname_result = [(ping_pattern.match(l1).group(1), l2) for l1, l2 in zip(hostname_result, hostname_result[1:]) if ping_pattern.match(l1)]
hostname_result

Docker Swarmの通信に使用するIPアドレス(VMのIPアドレス、インベントリの`servicenet_ip`)と、ホスト名の関係を調べる。

In [ ]:
%%writefile {temp_dir}/service_ip2hostname.yml
---
- become: true
  hosts: all
  tasks:
      - debug: "msg={{ansible_hostname}}={{servicenet_ip}}"


In [ ]:
import re

msgout = !ansible-playbook -l {target_group} {temp_dir}/service_ip2hostname.yml
msgout = msgout.grep('"msg"')
assert len(msgout) >= 1
hostname_result = []
for l in msgout:
    m = re.match(r'.*\"msg\": \"(.+)=(.+)\"', l)
    if m:
        hostname_result.append((m.group(2), m.group(1)))
hostname_result

現在のhostsを確認しておく

In [ ]:
!ansible -a 'cat /etc/hosts' {target_group}

設定する

In [ ]:
for ipaddr, hostname in hostname_result:
    !ansible -b -CDv -m lineinfile -a 'dest=/etc/hosts regexp="^{ipaddr} " line="{ipaddr} {hostname}"' {target_group}

In [ ]:
for ipaddr, hostname in hostname_result:
    !ansible -b -m lineinfile -a 'dest=/etc/hosts regexp="^{ipaddr} " line="{ipaddr} {hostname}"' {target_group}

hostsを確認しておく

In [ ]:
!ansible -a 'cat /etc/hosts' {target_group}

## Setting up the Master

まず、managerノードとなるDocker EngineのIPアドレスを取得する。

In [ ]:
master_ip_stdout = !ansible -m ping {target_master} -l {target_group}
manager_ip = [line.split()[0] for line in master_ip_stdout if 'SUCCESS' in line][0]
manager_ip

Docker Swarmを初期化する。

In [ ]:
!ansible -b -a 'docker swarm init --advertise-addr {manager_ip}' {target_master} -l {target_group}

上記セルの実行結果に、Swarm Tokenが表示されるので、これを以下の変数に控えておく。(IPアドレスも合わせて)

In [ ]:
# このセルを実行すると、テキストフィールドが現れるので、docker swarm join以降を入力する。
swarm_token = input()
if 'docker swarm join --token' in swarm_token.lstrip():
    swarm_token = swarm_token.lstrip()[len('docker swarm join --token'):].strip()
swarm_token

`docker info`で、`Swarm: active`となっていることを確認。

In [ ]:
!ansible -b -a 'docker info' {target_master} -l {target_group}

現在は1ノードで動作しているはず。

In [ ]:
!ansible -b -a 'docker node ls' {target_master} -l {target_group}

## Add nodes to the swarm

https://docs.docker.com/engine/swarm/swarm-tutorial/add-nodes/

In [ ]:
!ansible -b -a 'docker swarm join --token {swarm_token}' {target_nodes} -l {target_group}

ノードは追加されたか？

In [ ]:
!ansible -b -a 'docker node ls' {target_master} -l {target_group}

これでSwarmが構成されたはず

# 後始末

In [ ]:
!rm -rf {temp_dir}